# 02 — Scikit-learn Experimentation
Local experiments using scikit-learn.

In [15]:
import sagemaker
import boto3
import pandas as pd
import numpy as np

# --- 1. SageMaker Session & Role ---
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
prefix = 'titanic-ml' # Project prefix for S3 organization

print(f"SageMaker Role ARN: {role}")
print(f"S3 Bucket: {bucket}")
print(f"S3 Prefix: {prefix}")

SageMaker Role ARN: arn:aws:iam::323649454838:role/service-role/AmazonSageMaker-ExecutionRole-20251106T213876
S3 Bucket: sagemaker-ap-southeast-1-323649454838
S3 Prefix: titanic-ml


In [17]:
# --- Configuration (Run after the Prerequisites Setup block) ---
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.inputs import TrainingInput

# The S3 path where your processed data was saved in the previous notebook
INPUT_S3_URI = f"s3://{bucket}/{prefix}/processed"

# --- 1. Configure Estimator ---
sklearn_estimator = SKLearn(
    entry_point='model_sklearn.py',
    source_dir='../src', # Directory containing the script
    role=role,
    instance_count=1,
    instance_type='ml.c5.xlarge', # Use a suitable instance type
    framework_version='1.2-1', # Choose a supported Scikit-learn version
    py_version='py3',
    hyperparameters={
        'n_estimators': 200,
        'max_depth': 8
    },
    base_job_name='sagemaker-titanic-sklearn'
)

# --- 2. Prepare Data Input Channel ---
# The 'train' channel name must match the argument parser in your script
inputs = {
    'train': TrainingInput(
        s3_data=INPUT_S3_URI, 
        distribution='FullyReplicated', 
        content_type='text/csv', 
        s3_data_type='S3Prefix'
    )
}

# --- 3. Launch Training Job ---
print("Launching Scikit-learn training job in SageMaker...")
sklearn_estimator.fit(inputs, wait=False) # Use wait=True for synchronous execution
print(f"Training job launched: {sklearn_estimator.latest_training_job.job_name}")

INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.


Launching Scikit-learn training job in SageMaker...


INFO:sagemaker:Creating training-job with name: sagemaker-titanic-sklearn-2025-11-06-19-48-40-132


Training job launched: sagemaker-titanic-sklearn-2025-11-06-19-48-40-132


In [19]:
# Assuming the training job has completed (status is 'Completed')
# If the training job is still running, this cell will fail or hang if wait=True was used previously.

# --- 1. Define Endpoint Configuration ---
instance_type_for_endpoint = 'ml.m5.large' # Use a cost-effective instance for inference

# --- 2. Deploy the Model ---
print(f"Deploying model to a SageMaker Endpoint on {instance_type_for_endpoint}...")

# The deploy() method creates a Model, an EndpointConfig, and an Endpoint.
predictor = sklearn_estimator.deploy(
    initial_instance_count=1,
    instance_type=instance_type_for_endpoint,
    endpoint_name='sagemaker-titanic-sklearn-endpoint-v3' # Optional: name your endpoint
)

print("\n✨ Deployment Complete! Endpoint is ready for real-time predictions. ✨")
print(f"Endpoint Name: {predictor.endpoint_name}")

INFO:sagemaker:Creating model with name: sagemaker-titanic-sklearn-2025-11-06-19-52-45-957


Deploying model to a SageMaker Endpoint on ml.m5.large...


INFO:sagemaker:Creating endpoint-config with name sagemaker-titanic-sklearn-endpoint-v3
INFO:sagemaker:Creating endpoint with name sagemaker-titanic-sklearn-endpoint-v3


------!
✨ Deployment Complete! Endpoint is ready for real-time predictions. ✨
Endpoint Name: sagemaker-titanic-sklearn-endpoint-v3


## 📝 Important Notes

**About the Endpoint:**
- The `input_fn` in `model_sklearn.py` expects **CSV format** (`text/csv` content type)
- Make sure to set `predictor.serializer = CSVSerializer()` before making predictions
- Input data should be a 2D numpy array with shape `(n_samples, 11)` where n_samples is the number of passengers
- The 11 features are: Pclass, Age, Fare, FamilySize, Sex_male, Title_Mr, Title_Mrs, Title_Rare, Embarked_S, Embarked_Q, Embarked_C

**Feature Order (important!):**
1. Pclass (1, 2, or 3)
2. Age (float)
3. Fare (float)
4. FamilySize (int)
5. Sex_male (0 or 1)
6. Title_Mr (0 or 1)
7. Title_Mrs (0 or 1)
8. Title_Rare (0 or 1)
9. Embarked_S (0 or 1)
10. Embarked_Q (0 or 1)
11. Embarked_C (0 or 1)

In [ ]:
# --- Example Test Data (Must mimic the processed data format) ---
# Create a sample DataFrame with the correct features and data types
# This example uses placeholder values for the first row of your processed data features
sample_data = [[
    3,    # Pclass
    22.0, # Age
    7.25, # Fare
    1,    # FamilySize
    0,    # Sex_male
    0,    # Title_Mr
    0,    # Title_Mrs
    0,    # Title_Rare
    1,    # Embarked_S
    0,    # Embarked_Q
    0     # Embarked_C
]]

# IMPORTANT: Must send as CSV string, not numpy array
# The input_fn in model_sklearn.py expects 'text/csv' content type
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

# Configure predictor to use CSV format
predictor.serializer = CSVSerializer()
predictor.deserializer = JSONDeserializer()

# Convert to list (CSVSerializer will handle the conversion to CSV string)
payload = sample_data  # Send as list, not numpy array

# --- Send Prediction Request ---
print("Sending test data to the endpoint...")
print(f"Payload: {payload}")

response = predictor.predict(payload)

print(f"\n✅ Raw Prediction Response: {response}")
print(f"Prediction: {response[0]} ({'Survived' if response[0] == 1 else 'Did not survive'})")

Sending test data to the endpoint...


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:24                                                                                   │
│                                                                                                  │
│   21 # --- Send Prediction Request ---                                                           │
│   22 print("Sending test data to the endpoint...")                                               │
│   23                                                                                             │
│ ❱ 24 response = predictor.predict(payload)                                                       │
│   25                                                                                             │
│   26 print(f"Raw Prediction Response: {response}")                                               │
│   27                                                                                             │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/base_predictor.py:212 in predict               │
│                                                                                                  │
│   209 │   │   if inference_component_name:                                                       │
│   210 │   │   │   request_args["InferenceComponentName"] = inference_component_name              │
│   211 │   │                                                                                      │
│ ❱ 212 │   │   response = self.sagemaker_session.sagemaker_runtime_client.invoke_endpoint(**req   │
│   213 │   │   return self._handle_response(response)                                             │
│   214 │                                                                                          │
│   215 │   def _handle_response(self, response):                                                  │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/botocore/client.py:569 in _api_call                      │
│                                                                                                  │
│    566 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    567 │   │   │   │   )                                                                         │
│    568 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  569 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    570 │   │                                                                                     │
│    571 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    572                                                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/botocore/client.py:1023 in _make_api_call                │
│                                                                                                  │
│   1020 │   │   │   │   "Code"                                                                    │
│   1021 │   │   │   )                                                                             │
│   1022 │   │   │   error_class = self.exceptions.from_code(error_code)                           │
│ ❱ 1023 │   │   │   raise error_class(parsed_response, operation_name)                            │
│   1024 │   │   else:                                                                             │
│   1025 │   │   │   return parsed_response                                                        │
│   1026                                                     

In [ ]:
# --- Additional Test Cases ---
print("\n" + "="*60)
print("Testing Multiple Passengers")
print("="*60)

# Test Case 1: First class female (high survival probability)
test_1 = [[1, 30.0, 100.0, 1, 0, 0, 0, 0, 1, 0, 0]]
result_1 = predictor.predict(test_1)
print(f"\nTest 1 - First class female, age 30:")
print(f"  Features: {test_1[0]}")
print(f"  Prediction: {result_1[0]} ({'Survived ✓' if result_1[0] == 1 else 'Did not survive ✗'})")

# Test Case 2: Third class male (low survival probability)
test_2 = [[3, 25.0, 8.0, 0, 1, 1, 0, 0, 1, 0, 0]]
result_2 = predictor.predict(test_2)
print(f"\nTest 2 - Third class male, age 25:")
print(f"  Features: {test_2[0]}")
print(f"  Prediction: {result_2[0]} ({'Survived ✓' if result_2[0] == 1 else 'Did not survive ✗'})")

# Test Case 3: Second class female with family
test_3 = [[2, 35.0, 50.0, 3, 0, 0, 1, 0, 0, 1, 0]]
result_3 = predictor.predict(test_3)
print(f"\nTest 3 - Second class female with family, age 35:")
print(f"  Features: {test_3[0]}")
print(f"  Prediction: {result_3[0]} ({'Survived ✓' if result_3[0] == 1 else 'Did not survive ✗'})")

print("\n" + "="*60)
print("All tests completed!")
print("="*60)

In [ ]:
# --- WARNING: ONLY RUN THIS WHEN YOU ARE DONE TESTING ---
print(f"\nDeleting endpoint: {predictor.endpoint_name}...")
predictor.delete_endpoint(delete_endpoint_config=True)
print("Endpoint successfully deleted.")